In [36]:
## VISUALIZATION OF ALL DISTRIBUTION

In [170]:
MPOWER_GAIT_DATA_V1 = "syn10308918"
MPOWER_DEMO_DATA_V1 = "syn10371840"
MPOWER_GAIT_DATA_V2 = "syn21113231"
MPOWER_DEMO_DATA_V2 = "syn15673379"
MPOWER_GAIT_DATA_PASSIVE = "syn21114136"
EMS_PROF_DATA = "syn10235463"
EMS_DEMO_DATA = "syn10295288"
EMS_GAIT_DATA = "syn21256442"
EMS_PASSIVE_DATA = "syn10651116"

In [209]:
import pandas as pd
import synapseclient as sc
import sys
sys.path.append("../../src")
import utils
import preprocessing_utils as proc
from datetime import datetime
import numpy as np

In [28]:
syn = sc.login()

Welcome, aryton tediarjo!



In [103]:
## gait mPower V1 ##

demo_data = syn.tableQuery("SELECT * FROM {}".format(MPOWERV1_DEMO_DATA)).asDataFrame()
gait_data = syn.tableQuery("SELECT * FROM {}".format(MPOWERV1_GAIT_DATA)).asDataFrame()
demo_data = demo_data[["healthCode", "gender", "age",
                       "professional_diagnosis", "inferred_diagnosis"]].reset_index(drop = True)
data = pd.merge(gait_data, demo_data, on = "healthCode", how = "inner")
data = data.dropna(subset = ["inferred_diagnosis"], thresh = 1)
data["PD"] = data["inferred_diagnosis"].map({True :1.0, False:0.0})
data = data[(data["gender"] == "Female") | (data["gender"] == "Male")]
data["age"] = data["age"].apply(lambda x: float(x))
data = data[(data["age"] < 100) & (data["age"] > 0)]
data["gender"] = data["gender"].apply(lambda x: x.lower())
data = data[["healthCode", "age", "gender", "PD"]] \
        .drop_duplicates(subset = "healthCode", keep = "first").reset_index(drop = True)
utils.save_data_to_synapse(data = data, output_filename = "mpower_v1_full_metadata.csv",
                          data_parent_id = "syn21267355")


##################################################
 Uploading file to Synapse storage 
##################################################



In [194]:
def create_mPowerV2_data(MPOWER_DATA_V2, MPOWER_DEMO_V2, output_filename):

    demo_data = syn.tableQuery("SELECT birthYear, healthCode, diagnosis, sex FROM {} \
                                where dataGroups NOT LIKE '%test_user%'".format(MPOWER_DEMO_V2)).asDataFrame()
    gait_data = utils.get_file_entity(MPOWER_DATA_V2)
    data   = pd.merge(gait_data, demo_data, how = "inner", on = "healthCode")
    data   = data[data["diagnosis"] != "no_answer"] 
    data["PD"] = data["diagnosis"].map({"parkinsons":1, "control":0})
    data["age"] = data["birthYear"].apply(lambda year: datetime.now().year - year)
    data = data.rename({"sex":"gender"}, axis = 1)
    data = utils.fix_column_name(data)
    data = data.reset_index(drop = True)
    utils.save_data_to_synapse(data = data, output_filename = output_filename,
                              data_parent_id = "syn21267355")
    return data

In [195]:
def create_elevateMS_data(EMS_DATA, EMS_PROF_DATA, output_filename):
    profile_data = syn.tableQuery("SELECT healthCode, dataGroups, 'demographics.gender', 'demographics.age' FROM {}\
                           where dataGroups NOT LIKE '%test_user%'".format(EMS_PROF_DATA)).asDataFrame()
    gait_data    = utils.get_file_entity(EMS_DATA)
    data         = pd.merge(gait_data, profile_data, how = "inner", on = "healthCode")
    data = data.dropna(subset = ["demographics.gender"])
    data["MS"] = data["dataGroups"].map({"ms_patient":1, "control":0})
    data  = data.rename({"demographics.gender" :"gender",
                         "demographics.age"    : "age"}, axis = 1)
    data = utils.fix_column_name(data)
    data = data.reset_index(drop = True)
    utils.save_data_to_synapse(data = data, output_filename = output_filename,
                                data_parent_id = "syn21267355")
    return data

In [196]:
dataV2                    = create_mPowerV2_data(MPOWER_GAIT_DATA_V2, MPOWER_DEMO_DATA_V2, "pdkit_mpowerv2_active_full.csv")
dataV2["version"]         = "V2"
dataPassive               = create_mPowerV2_data(MPOWER_GAIT_DATA_PASSIVE, MPOWER_DEMO_DATA_V2, "pdkit_mpowerv2_passive_full.csv")
dataPassive["version"]    = "Passive"
dataEMS_active            = create_elevateMS_data(EMS_GAIT_DATA, EMS_PROF_DATA, "pdkit_ems_active_full.csv")
dataEMS_active["version"] = "EMS_active"


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################



In [214]:
### some data annotations ### 
data = pd.concat([dataV2, dataPassive, dataEMS_active]).reset_index(drop = True)
metadata_cols = ['recordId', 'healthCode', 'appVersion', 
                 'phoneInfo', 'createdOn', 'PD', 'MS',
                 'gender', 'age', 'version']
data = data[[feat for feat in data.columns if ("." in feat) or (feat in metadata_cols)]]
data = data[(data != "#ERROR").all(axis = 1)]
data["is_control"] = data.apply(lambda x: 0 if ((x["PD"] == 0) or (x["MS"] == 0)) else 1, axis = 1)
data[[_ for _ in data.columns if "." in _]] = \
    data[[_ for _ in data.columns if "." in _]].apply(pd.to_numeric)
data.drop(["y.duration", "z.duration", "AA.duration"], axis = 1, inplace = True) 
data.rename({"x.duration": "duration"}, axis = 1, inplace = True)
data = proc.addAdditionalFeatures().transform(data)
# data = proc.collapseFeatures(aggregation_type = "mean").transform(data)
utils.save_data_to_synapse(data = data.reset_index(drop = True), output_filename = "combined_gait_data.csv",
                                data_parent_id = "syn21267355")

[WARNING] /Users/atediarjo/Documents/SageBionetworks/synapse_venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  




##################################################
 Uploading file to Synapse storage 
##################################################

